# Figure module
繪圖相關

In [1]:
import matplotlib.pyplot as plt
import plotly.graph_objs as pltgo
import numpy as np
import pandas as pd

import import_ipynb
from transaction import get_transaction_DF

importing Jupyter notebook from transaction.ipynb
importing Jupyter notebook from helper_func.ipynb
Warning - Certain functionality 
             requires requests_html, which is not installed.
             
             Install using: 
             pip install requests_html
             
             After installation, you may have to restart your Python session.
importing Jupyter notebook from profit_loss.ipynb


## Bollinger Bands

In [2]:
# DF: Company's historical datas
def show_bollinger(DF, ticker, start_date, end_date, SIG=None):
    plt.figure(figsize=(20,10))
    plt.plot(DF['upper'], color='#CC99FF')
    plt.plot(DF['middle'], color="blue")
    plt.plot(DF['lower'], color='#CC99FF')
    plt.plot(DF['close'], color="black")
    if SIG is not None:
        plt.plot(DF['close'][SIG == 1], lw = 0, marker='^', markersize=10, c='g', alpha=0.7)
        plt.plot(DF['close'][SIG == 2], lw = 0, marker='v', markersize=10, c='r', alpha=0.7)
    plt.xlabel('date')
    plt.ylabel('price')
    plt.legend(['upper', 'middle', 'lower', 'close'])
    plt.grid(True)
    plt.title(f"{ticker} Bollinger band {start_date} ~ {end_date}")
    plt.show()

In [3]:
# Advance way to show bollinger bands
def bollinger_bands(DF, ticker):
    fig = pltgo.Figure()
    
    # Setup traces
    fig.add_trace(pltgo.Scatter(x=DF.index, y=DF['upper'], line=dict(color='yellow', width=1.7), name = 'Upper Band'))
    fig.add_trace(pltgo.Scatter(x=DF.index, y=DF['middle'], line=dict(color='blue', width=.7), name = 'Middle Band'))
    fig.add_trace(pltgo.Scatter(x=DF.index, y=DF['lower'], line=dict(color='yellow', width=1.7), name = 'Lower Band'))
    fig.add_trace(pltgo.Scatter(x=DF.index, y=DF['close'], line=dict(color='black', width=.7), name = 'Close price'))
    
    # candle stick
    fig.add_trace(pltgo.Candlestick(x=DF.index, open=DF['open'], high=DF['high'],
                                    low=DF['low'], close=DF['close'], name="market data"))
    fig.update_layout(title=ticker)
    fig.show()

## Return

In [4]:
# 顯示回測
def show_returns(df):
    plt.hist(df['returns'].dropna(),bins=75,density=False)
    plt.show

## Funding Changes

In [5]:
# draw funding changes
def funding_changes(rec):
    rec= rec[rec['動作']=='SELL']
    plt.figure(figsize=(25, 10))
    plt.plot(rec['日期'], rec['總資金'],color="green")
    plt.title("IGS Funding changes (principal: 300000)")
    plt.show()

## Profit Loss

In [6]:
# 畫2D 勝率& 損益比
# tickers: 股票代碼list
# colors: 指定顏色
# basic_ratio: 看你想分析多少倍的停損停利
def draw_2D_WL(tickers, basic_ratio=None, colors = ['b', 'c', 'y', 'm', 'r'], max_pw = 0.8 ):

    # 基本圖表設置
    plt.figure(figsize=(10,10))
    plt.xlabel('Pw')
    plt.ylabel('W/L')

    # 畫出期望線
    exp_pw = np.linspace(0.1, max_pw)  # 假設勝率分布在0.1~max_pw
    exp_WL = (1-exp_pw) / exp_pw  # 算出公平WL
    plt.plot(exp_pw ,exp_WL , '-', c='g')
    
    # 畫出股票點
    for i,ticker in enumerate(tickers):
        df = get_transaction_DF(ticker, basic_ratio=basic_ratio)
        plt.plot( df['Pw'] , df['W/L'] , 'o' ,c=colors[i] , label=ticker )
    plt.legend()
    plt.show()

In [7]:
# 畫2D 勝率& 損益比
# tickers: 股票代碼list
# basic_ratio: 看你想分析多少倍的停損停利
def draw_bubble_WL(tickers , basic_ratio=None, colors = ['b', 'c', 'y', 'm', 'r'] , max_pw = 0.8):

    # 基本圖表設置
    plt.figure(figsize=(15,15))
    plt.xlabel('Pw')
    plt.ylabel('W/L')# 畫出期望線
    exp_pw = np.linspace(0.1, max_pw)  # 假設勝率分布在0.1~max_pw
    exp_WL = (1-exp_pw) / exp_pw  # 算出公平 WL
    plt.plot(exp_pw ,exp_WL , '-', c='g')# 畫出股票點
    for i,ticker in enumerate(tickers):
        df = get_transaction_DF(ticker,basic_ratio=basic_ratio)
        norm = np.linalg.norm(df['投資報酬率(%)'])
        plt.scatter( df['Pw'] , df['W/L'], s= (0.2+(df['投資報酬率(%)']/norm))*3000  ,c=colors[i] , label=ticker , alpha=0.6 )
        plt.legend()
    plt.show()

In [8]:
# 畫3D直方圖
# ticker: 股票代碼
# basic_ratio: 看你想分析多少倍的停損停利
# dx_rate: 正方體 x軸寬度，視結果調整
# dy_rate: 正方體 y軸寬度，視結果調整
def draw_PL_BEP_Figure(ticker, basic_ratio=None, dx_rate = 0.01, dy_rate = 0.05):

    df = get_transaction_DF(ticker, basic_ratio=basic_ratio)
    
    figure = plt.figure(figsize=(12, 8))
    axes = figure.add_subplot(111,projection = "3d")
    xpos = df['Pw']
    ypos = df['W/L']
    zpos = np.zeros(len(df['投資報酬率(%)']))
    dx = np.ones(len(df['Pw']))
    for i in range(len(dx)):
        dx[i] = dx_rate
    dy = np.ones(len(df['W/L']))
    for i in range(len(dy)):
        dy[i] = dy_rate
    dz = df['投資報酬率(%)']
    axes.bar3d(xpos, ypos, zpos, dx, dy, dz, color='red')
    axes.set_xlabel('Pw')
    axes.set_ylabel('W/L')
    axes.set_zlabel('hpr%')

    plt.show()
    return df